# Lesson7 ニューラルネットでゲームを攻略するAIをつくろう

## 目次

- Section 1 解説
    - 1.1 強化学習とは
    - 1.2 強化学習の定式化
    - 1.3 Temporal Difference (TD) 学習
    - 1.4 Q学習
    - 1.5 Deep Q学習
    - 1.6 Checkクイズ
- Section 2 実装①
    - 2.1 OpenAI Gymについて
    - 2.2 Colab上でのレンダリングについて
    - 2.3 DQNでCartPole
- Section 3 テクニック・発展的内容
    - 3.1 Prioritized Experience Replay
    - 3.2 Double DQN
    - 3.3 Dueling Network
    - 3.4 Checkクイズ
- Section 4 実装②
    - 4.1 DQN + テクニックでBreakout
- Section 5 ケーススタディ

## Section 1 解説

### 1.1 強化学習とは

強化学習とは、エージェントが環境と試行錯誤しながらより良い行動の仕方を学習していく方法です。教師あり学習のように明示的に教師が与えられず、報酬という間接的な目標からのフィードバックを元に学習していきます。

いくつか具体的な例を見てみます。

#### 1.1.1 具体的な例

#### スマホの電波が良い場所を探す
スマホの電波が悪くて、電波の良いスポットに移動したいと思うことはしばしばあるかと思います。

このとき私達は(エージェント)は、スマホの電波が良い場所というのはわからず、さぐりさぐり歩きながら電波が良くなりそうな方向に歩いていきます。

つまり、教師のようなもの(電波maxなスポットがどこか)というのは分からない一方、電波の良し悪しはスマホの画面をみることで幾分測ることができます。

出典: J. Si et al. Handbook of Learning and Approximate Dynamic Programming. Wiley-IEEE Press. 2004

#### お金持ちになる
人生の目的の１つにお金持ちになるということがあげられるかと思います。

この目的に到達するために、私達(エージェント)はいい職につくためにいい学校に入ろうとしたり、年収を上げるために転職したりします。同時に、その過程々々でその時点での所得がどれくらいなのかを測りながら、その都度行動を修正していきます。

また、ある地点で所得が上がりそうでない行動をした(e.g. 留年した、失職した)場合でも、後々それが良い結果につながったのではないかと考えられることも度々あります。

#### 1.1.2 強化学習で取り扱う問題の性質

これらの例は強化学習で扱うべき問題の典型例であり、以下のような重要な要素が含まれています。

- 明確な教師が存在しない
- 結果(報酬)が遅れて反映される
- 探索と活用のトレードオフがある
- 観測するデータが取った行動に依存する

強化学習は教師あり学習と上記のような点で異なります。

### 1.2 強化学習の定式化

強化学習では次のように定式化を行います。

行動する主体(Agent)が現在の状態(State)にもとづいて行動(Action)を選択し、それを環境(Environment)が受け取り、新しい状態が生成され、Agentはそれを元にまた行動を選択するというプロセスです。

上のスマホの電波の例で言えば、エージェントは私達人間、行動はどの向きにどれくらい動くか、環境はスマホの電波マップ、報酬はその地点での電波の良し悪しという感じになります。

また以下では時刻$t+1$における状態$S_{t+1}$及び報酬$R_{t+1}$は、時刻$t$における状態$S_t$及び行動$A_t$のみに依存することを仮定して考えます。これを**マルコフ性**を仮定するといいます。

Q学習、DQNなどはこのマルコフ性を仮定した下でのアルゴリズムです。

一見マルコフ性を満たしていない環境、例えばブロック崩しの静止画だけでは玉がどの向きに動いているかわからないような状況においても、複数フレームをstackなどすれば問題ありません。

<img src="./figures/rl_concept.png" width="700mm">

出典: Richard S. Sutton and Andrew G. Barto. 2017. Reinforcement Learning: An Introduction. Second edition. The MIT Press

またこの一連の繰り返しを **エピソード** と呼びます。

お金持ちの例で言えば1人生が1エピソードです。

エピソード的でない(終わりがない)タスクもありますが、ここでは扱いません。

#### 1.2.1 強化学習における目的

エージェントとしては、最終的に良い結果を得るためにどのような行動を取ったらいいかを学習していきます。この「どのような行動を取るか」についてのエージェントの指針を強化学習では**方策 (policy、 $\pi$)**といい、いかに良い方策を学習できるががポイントになります。

具体的には、方策は状態を入力とし行動を出力とする関数となります。

強化学習では方策の良さを累積の利得で表現します。これを**利得**と呼びます。

$$
    G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots = \sum^{\infty}_{k=0} \gamma^k R_{t+k+1}
$$
ここで$\gamma$は割引率と呼ばれ、将来の報酬に対してどれだけ割り引くのかの度合いを表します。

この利得(累積報酬)の期待値を最大化するような方策を求めることが強化学習の目的となります。

式で表すと次のようになります

$$
    \pi^* = \mathrm{arg}\max_{\pi}\mathbb{E}_{\pi}[\sum^{\infty}_{k=0}\gamma^k R_{t+k+1}]
$$

この$\pi^*$を**最適方策**と呼びます。

### 1.3 Temporal Difference (TD) 学習

最適方策をもとめるアプローチにはいくつかありますが、ここではTD学習と呼ばれる、価値関数の推定に基づくものを紹介していきます。これはDQNのもとになっている考え方です。

まず価値関数とは何かについて説明し、そこから最適方策へのつなげ方に移っていきます。

#### 1.3.1 価値関数

価値関数とは、ある方策$\pi$の下で、各状態にいたとき・各行動をとったときに将来どれくらい利得を得ることができるかを示すもので、それぞれ**状態価値関数**、**行動価値関数**と呼ばれます。

#### 状態価値関数
エージェントが状態$s$にいたとき、方策$\pi$の下で将来得られる利得の期待値を表します。
$$
    v_{\pi}(s) = \mathbb{E}_{\pi}[R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \ldots|S_t = s] = \mathbb{E}_{\pi}[\sum^{\infty}_{k=0}\gamma^k R_{t+k+1}|S_t=s]
$$

#### 行動価値関数
エージェントがある状態$s$において行動$a$をとったとき、方策$\pi$の下で将来得られる利得の期待値を表します。
$$
    q_{\pi}(s,a) = \mathbb{E}_{\pi}[R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \ldots|S_t=s, A_t=a] = \mathbb{E}_{\pi}[\sum^{\infty}_{k=0}\gamma^k R_{t+k+1}|S_t=s, A_t=a]
$$

#### 1.3.2 価値関数の更新

ここで、行動価値関数が次のように表すことができることに注目します。(状態価値関数でも同様)

$$
    q_{\pi} = \mathbb{E}_{\pi}[G_t|S_t=s, A_t=a] = \mathbb{E}_{\pi}[R_{t+1} + \gamma q_{\pi}(S_{t+1})|S_t=s, A_t=a]
$$

つまり状態$s$における価値関数は、その次のステップで得た報酬とその次の状態$s'$の価値関数に割引を掛けた和$R_{t+1} + \gamma q_{\pi}(S_{t+1})$で表すことができます。

ここで、**後者の方が実際の報酬$R_{t+1}$を観測している分真の価値関数に近い**と考え、現在の推定値$Q_{\pi}(s)$と新しい推定値$R_{t+1} + \gamma Q_{\pi}(s')$を利用して以下のように更新を行います。

$$
    Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \left[R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)\right]
$$

$\alpha$は更新幅を決めるハイパーパラメータです。

行動価値関数の場合は特に**Sarsa**と呼ばれます。

ここで両式のカッコの中身(新しい推定値 - 古い推定値)は **TD誤差** と呼ばれます。

このように、**推定値からその時点での推定値をもとに更新を行う**ために、**Temporal Difference (TD) 学習**と呼ばれます。

これがTD学習の重要なポイントで、更新のターゲットに推定値を利用するので、エピソードが終結するのを待たずに価値関数を更新することができます。

#### 1.3.3 方策の更新

前述の通り、強化学習の最終目的は最適方策を求めることです。

そのため、価値関数の更新をおこなったあと、その価値関数のもとで各状態・行動において利得を最大化することができるように方策を更新していきます。

具体的には各状態において最も期待利得が大きくなるような行動を取る(greedy)ように更新を行います。

イメージとしては次の図のようになります。
価値関数の更新と方策の更新を繰り返していきます。

<img src="./figures/gpi.png" width="300mm">

出典: Richard S. Sutton and Andrew G. Barto. 2017. Reinforcement Learning: An Introduction. Second edition. The MIT Press

### 1.4 Q学習

Q学習もTD学習の一種でありますが、少し違います。

Sarsaではその時点での方策の価値関数を求めるよう更新式を設定していましたが、Q学習では直接最適方策をもとめるように更新を行います。

つまり、Sarsaでは価値関数の更新->方策の更新->価値関数の更新->方策の更新->...と繰り返していましたが、Q学習では最適方策を直接ターゲットとして学習するため、価値関数の更新のみを考えれば良いということです。

具体的な更新式は次のようになり、新しい推定値の計算をgreedyな方策の下で行うことで、最適方策における価値関数に直接近づけていきます。

$$
    Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \left[R_{t+1} + \gamma \max_a Q(S_{t+1}, a) - Q(S_t, A_t)\right]
$$

そのため、Sarsaでは価値推定する方策と探索に使う方策は同じでしたが、Q学習では価値推定する方策と探索に使う方策は違って良いということになります。

Sarsaのように価値推定する方策と探索につかう方策が同じ必要があるアルゴリズムを **on-policy**、Q学習のように同じ必要がないアルゴリズムを **off-policy** といいます。

Q学習では探索と活用のトレードオフを実現するため、ある一定の確率$\epsilon$ではランダムに行動し、$1-\epsilon$でその時の最もよい行動を選択する **$\epsilon$-greedy方策** を探索に用いることが一般的です。

### 1.5 Deep Q学習

今回のメインテーマとなるDeep Q-Network (DQN)は、このQ学習をDeep Learningで上手くおこなおうというものです。

DQNでは、状態を入力とし各行動の価値関数を出力とするネットワークを構築します。このネットワークが **Q-Network** と呼ばれます。イメージは次の図のようになります。

<img src="./figures/dqn.png" width="500mm">

出典: V. Mnih et al. Human-level control through deep reinforcement learning. Nature. 2015

Q学習で用いたTD誤差を2乗したものを教師あり学習における誤差関数と考え、通常の勾配法などで最適方策を求めていきます。

- 誤差関数
$$
    \mathcal{L} = \mathbb{E}[(R_{t+1} + \gamma \max_{a'}Q(S_{t+1}, a') - Q(S_t, A_t))^2]
$$

ここで、教師あり学習のように学習をすすめるための工夫として、次の2つを導入します。

#### 1.5.1 Experience Replay

前述の通り、強化学習ではエージェントが取った行動によって次の状態・報酬が変化してきてしまうため、学習につかうサンプルの分布が安定しません。

DQNでは、エージェントは行動を進めつつ、実際の学習には過去それまでのデータからランダムにサンプリングしたものをパラメータの更新に使用します。こうすることでサンプリングされるデータの分布を安定させます。過去の経験から思い出すという意味で、これを**Experience Replay**と言います。

#### 1.5.1 Target Network

通常の教師あり学習では、教師信号が固定された状況で、いかにそれにネットワークの出力を近づけるかという思いでパラメータを更新していました。
ただ前述の通り、通常のQ学習では**推定値から推定を行う**という、教師信号が固定されないすこしこれとは異なった状況でした。

**Target Network**では、Q学習における教師信号、つまり新しい推定値($R_{t+1} + \gamma \max_{a'}Q(s',a')$)を一定ステップ固定することでこの問題の軽減を試みます。
具体的には、通常のQ-Networkともう一つ同じ構造のTarget-Networkを保持し、一定ステップごとにQ-NetworkのパラメータをTarget-Networkにコピーすることでこれを実現します。

#### \* 記法整理
今Lessonで登場する記号を以下にまとめます。

- $G_t$: 利得. 時刻t以降に獲得できる報酬の累積値
- $\gamma$: 割引率
- $S_t$: 時刻$t$における状態
- $A_t$: 時刻$t$における行動
- $R_{t+1}$: 時刻$t+1$における報酬
- $v_{\pi}(s)$: 状態$s$における方策$\pi$の真の状態価値関数
- $V_{\pi}(s)$: 状態$s$における方策$\pi$の推定状態価値関数
- $q_{\pi}(s, a)$: 状態と行動のペア($s, a$)における方策$\pi$の真の行動価値関数
- $Q_{\pi}(s, a)$: 状態と行動のペア($s, a$)における方策$\pi$の推定行動価値関数
- $A_{\pi}(s, a)$: 状態と行動のペア($s, a$)における方策$\pi$のアドバンテージ関数
- $y^{(i)}$: 学習ステップ$i$におけるQ学習の教師信号
- $\delta^{(i)}$: 学習ステップ$i$におけるTD誤差
- $\pi$: 方策
- $\alpha$: 学習のステップ幅

### 1.6 Checkクイズ

問1. **行動価値関数$Q_{\pi}(s,a)$** に関する説明として正しいものを選択肢から一つ選びなさい。

① ある方策$\pi$の下で、 状態$s$で行動$a$をとったときそれ以降に獲得できる利得の期待値を表す。  
② ある方策$\pi$の下で、 状態$s$で行動$a$をとったときその次のステップで獲得できる報酬の期待値を表す。  
③ 最適方策$\pi^*$の下で、 状態$s$で行動$a$をとったときそれ以降に獲得できる利得の期待値を表す。  
④ 最適方策$\pi^*$の下で、 状態$s$で行動$a$をとったときその次のステップで獲得できる報酬の期待値を表す。

問2. **TD学習** に関する説明として正しいものを選択肢から一つ選びなさい。
  
① エピソードが終了したのち、 過去の状態・行動における利得を再帰的に計算して価値関数の更新をおこなっていく手法。  
② エピソードの各ステップにおいて、 そのステップにおける推定価値により価値関数の更新をおこなっていく手法。  
③ 状態の遷移確率を推定することで最適方策を求める手法。  
④ 状態の遷移確率が与えられている下で、 各ステップにおける推定価値により価値関数の更新をおこなっていく手法。  

問3. **Experience Replay** に関する説明として正しいものを選択肢から一つ選びなさい。

① 過去の履歴から経験をサンプリングし、 それを学習のデータとして使う手法。  
② 行動の各ステップで、 過去の履歴の中で同じ状態で高い報酬を得た行動を真似る手法。  
③ 過去に学んだ事を忘れないよう、 ある状態では一定回数以上同じ行動を取り続ける手法。  
④ Networkが過去の経験を重視しすぎて過学習してしまう問題。  